# CNN Model Architecture for Music Tagging

This notebook implements Convolutional Neural Network architectures for music classification and tagging.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio

## Simple CNN for Genre Classification

In [ ]:
class SimpleCNN(nn.Module):
    
    def __init__(self, n_classes=10, sample_rate=22050, n_mels=128):
        super(SimpleCNN, self).__init__()
        
        self.sample_rate = sample_rate
        self.n_mels = n_mels
        
        # Mel-spectrogram transform
        self.mel_spec = torchaudio.transforms.MelSpectrogram(
            sample_rate=sample_rate,
            n_fft=2048,
            hop_length=512,
            n_mels=n_mels
        )
        self.amplitude_to_db = torchaudio.transforms.AmplitudeToDB()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(2, 2)
        
        # Global average pooling
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Fully connected layers
        self.fc1 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, n_classes)
    
    def forward(self, x):
        # x: (batch, 1, time)
        
        # Force spectrogram computation in float32 to avoid numerical instability in AMP
        with torch.amp.autocast(device_type="cuda", enabled=False):
            x = x.float()
            x = self.mel_spec(x)
            x = self.amplitude_to_db(x)
        
        # x: (batch, 1, n_mels, time)
        
        # Convolutional blocks
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.pool4(F.relu(self.bn4(self.conv4(x))))
        
        # Global pooling and flatten
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

### Complex CNN using resnet architecture

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, dropout=0.0):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(dropout)
        self.conv2 = nn.Conv2d(
            out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_channels, out_channels, kernel_size=1, stride=stride, bias=False
                ),
                nn.BatchNorm2d(out_channels),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ComplexCNN(nn.Module):
    def __init__(self, n_classes=10, sample_rate=22050, n_mels=128):
        super(ComplexCNN, self).__init__()
        
        self.sample_rate = sample_rate
        self.n_mels = n_mels
        
        # Mel-spectrogram transform
        self.mel_spec = torchaudio.transforms.MelSpectrogram(
            sample_rate=sample_rate,
            n_fft=2048,
            hop_length=512,
            n_mels=n_mels
        )
        self.amplitude_to_db = torchaudio.transforms.AmplitudeToDB()
        
        # SpecAugment - Moderate masking
        self.freq_masking = torchaudio.transforms.FrequencyMasking(freq_mask_param=30)
        self.time_masking = torchaudio.transforms.TimeMasking(time_mask_param=40)

        # Initial Conv
        self.conv1 = nn.Conv2d(1, 32, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # ResNet Layers
        # Increased capacity: Added layer4 and kept dropout
        self.layer1 = self._make_layer(32, 64, 2, stride=1, dropout=0.3)   # 32 -> 64
        self.layer2 = self._make_layer(64, 128, 2, stride=2, dropout=0.3)  # 64 -> 128
        self.layer3 = self._make_layer(128, 256, 2, stride=2, dropout=0.3) # 128 -> 256
        self.layer4 = self._make_layer(256, 512, 2, stride=2, dropout=0.3) # 256 -> 512

        # Global Average Pooling and Classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, n_classes) # Input dim is now 512
        self.dropout = nn.Dropout(0.5)

    def _make_layer(self, in_channels, out_channels, blocks, stride, dropout=0.0):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride, dropout))
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels, dropout=dropout))
        return nn.Sequential(*layers)

    def forward(self, x):
        # x: (batch, 1, time)
        
        # Spectrogram computation
        with torch.no_grad():
             # Force float32 for stability
            with torch.amp.autocast(device_type="cuda", enabled=False):
                x = x.float()
                x = self.mel_spec(x)
                x = self.amplitude_to_db(x)
        
        # Apply SpecAugment only during training
        if self.training:
            x = self.freq_masking(x)
            x = self.time_masking(x)
            
        # CNN
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x) 

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)

        return x